In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_csv('poke_genie_export.csv')

In [3]:
df = df[df['Name'] == 'Phantump']

In [4]:
df.columns

Index(['Index', 'Name', 'Form', 'Pokemon Number', 'Gender', 'CP', 'HP',
       'Atk IV', 'Def IV', 'Sta IV', 'IV Avg', 'Level Min', 'Level Max',
       'Quick Move', 'Charge Move', 'Charge Move 2', 'Scan Date',
       'Original Scan Date', 'Catch Date', 'Weight', 'Height', 'Lucky',
       'Shadow/Purified', 'Favorite', 'Dust', 'Rank % (G)', 'Rank # (G)',
       'Stat Prod (G)', 'Dust Cost (G)', 'Candy Cost (G)', 'Name (G)',
       'Form (G)', 'Sha/Pur (G)', 'Rank % (U)', 'Rank # (U)', 'Stat Prod (U)',
       'Dust Cost (U)', 'Candy Cost (U)', 'Name (U)', 'Form (U)',
       'Sha/Pur (U)', 'Marked for PvP use'],
      dtype='object')

In [5]:

def get_cpm(level):
    d = {
        1:	0.094,
        1.5: 0.1351374318,
        2: 0.16639787,
        2.5: 0.192650919,
        3: 0.21573247,
        3.5: 0.2365726613,
        4: 0.25572005,
        4.5: 0.2735303812,
        5: 0.29024988,
        5.5: 0.3060573775,
        6: 0.3210876,
        6.5: 0.3354450362,
        7: 0.34921268,
        7.5: 0.3624577511,
        8: 0.3752356,
        8.5: 0.387592416,
        9: 0.39956728,
        9.5: 0.4111935514,
        10: 0.4225,
        10.5: 0.4329264091,
        11: 0.44310755,
        11.5: 0.4530599591,
        12: 0.4627984,
        12.5: 0.472336093,
        13: 0.48168495,
        13.5: 0.4908558003,
        14: 0.49985844,
        14.5: 0.508701765,
        15: 0.51739395,
        15.5: 0.5259425113,
        16: 0.5343543,
        16.5: 0.5426357375,
        17: 0.5507927,
        17.5: 0.5588305862,
        18: 0.5667545,
        18.5: 0.5745691333,
        19: 0.5822789,
        19.5: 0.5898879072,
        20: 0.5974,
        20.5: 0.6048236651,
        21: 0.6121573,
        21.5: 0.6194041216,
        22: 0.6265671,
        22.5: 0.6336491432,
        23: 0.64065295,
        23.5: 0.6475809666,
        24: 0.65443563,
        24.5: 0.6612192524,
        25: 0.667934,
        25.5: 0.6745818959,
        26: 0.6811649,
        26.5: 0.6876849038,
        27: 0.69414365,
        27.5: 0.70054287,
        28: 0.7068842,
        28.5: 0.7131691091,
        29: 0.7193991,
        29.5: 0.7255756136,
        30: 0.7317,
        30.5: 0.7347410093,
        31: 0.7377695,
        31.5: 0.7407855938,
        32: 0.74378943,
        32.5: 0.7467812109,
        33: 0.74976104,
        33.5: 0.7527290867,
        34: 0.7556855,
        34.5: 0.7586303683,
        35: 0.76156384,
        35.5: 0.7644860647,
        36: 0.76739717,
        36.5: 0.7702972656,
        37: 0.7731865,
        37.5: 0.7760649616,
        38: 0.77893275,
        38.5: 0.7817900548,
        39: 0.784637,
        39.5: 0.7874736075,
        40: 0.7903,
        40.5: 0.792803968,
        41: 0.79530001,
        }
    return d[level]

In [6]:
get_cpm(2)

0.16639787

In [19]:
# for trev
def ivs_to_stats(my_a, my_d, my_s,max_level=40,max_cp=1500.99):
    base_attack = 201
    base_defense = 154
    base_stamina = 198
    attack = base_attack + my_a
    defense = my_d + base_defense
    stamina = my_s + base_stamina
    level = 10
    cp = 10
    level_attack, level_defense, level_stamina = 0,0,0
    stats = (level, cp, level_attack, level_defense, level_stamina)
    while level <= max_level:
        cpm = get_cpm(level)
        cp = (attack * defense**0.5 * stamina**0.5 * cpm**2) / 10
        level_attack = attack * cpm
        level_defense = defense * cpm
        level_stamina = stamina * cpm
        if cp <= max_cp:
            stats = level, cp, level_attack, level_defense, level_stamina
        level = level + 0.5
    level, cp, level_attack, level_defense, level_stamina = stats
    return stats
    #print(f'level {level} cp {cp:.0f} attack {level_attack:.1f} defense {level_defense:.1f} stamina {level_stamina:.0f}')
    


In [20]:
ivs_to_stats(13,14,15)

(20.5, 1480.8660172622085, 129.4322643314, 101.6103757368, 128.8274406663)

In [21]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV']
    level, cp, attack, defense, stamina = ivs_to_stats(my_a, my_d, my_s,max_level=41)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
pht = pd.DataFrame.from_dict(d)

In [22]:
# gfisk attack bp
these = pht[pht.attack >= 167.98]
# hp breakpoint for mirror
these = these[these.stamina >= 167]
these

,CP,max CP,level,attack,defense,stamina,a,d,s


In [23]:
# mew attack bp
these = pht[pht.attack >= 127.8]
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these # 3/15/7 is RS, I think maybe 3/14/11 for Azu

,CP,max CP,level,attack,defense,stamina,a,d,s
17,570,1490,22.0,127.819688,105.889840,128,3,15,7
23,288,1500,22.0,127.819688,105.263273,130,3,14,11


In [13]:
# mew attack bp
these = pht[pht.attack >= 127] 
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these

,CP,max CP,level,attack,defense,stamina,a,d,s
17,570,1490,22.0,127.819688,105.88984,128,3,15,7


In [18]:
ivs_to_stats(my_a=3,my_d=14,my_s=11,max_cp=1501)

(22.0, 1500.697383945778, 127.8196884, 105.26327280000001, 130.95252390000002)